In [1]:
import pandas as pd, numpy as np
from db_queries import get_ids, get_outputs, get_location_metadata, get_population, get_covariate_estimates
from get_draws.api import get_draws
import matplotlib.pyplot as plt
import os

The purpose of this notebook is to estimate the proportion of smoking-attributable mortality that is due to lung cancer, COPD, and IHD in China in order to assess the magnitude of the limitation of our model in using these three causes as a proxy for all smoking-attributable mortality for the lung cancer screening model

In [2]:
smok_attrib = get_draws(gbd_id_type='rei_id',
                gbd_id=99,#426,493,509], #smoking, all causes, lung cancer, IHD, COPD
                source='burdenator',
                measure_id=1, # deaths
                metric_id=1, # count
                location_id=6, # china
                year_id=2019,
                gbd_round_id=6,
                decomp_step='step5',
                status='best')

smok_attrib = smok_attrib.loc[smok_attrib.cause_id.isin([294,426,493,509])]
smok_attrib.head()

,age_group_id,cause_id,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,...,draw_996,draw_997,draw_998,draw_999,location_id,measure_id,metric_id,rei_id,sex_id,year_id
0,11,294,466.560914,440.030820,463.484026,368.750515,356.996887,315.822957,373.557650,325.546755,...,574.119698,316.655890,351.343638,309.117415,6,1,1,99,2,2019
1,12,294,694.420240,378.672623,733.806222,605.367953,758.976669,762.299630,855.913949,797.127632,...,868.578567,659.880376,570.804420,715.466889,6,1,1,99,2,2019
2,13,294,1580.537425,1196.457942,1391.908416,2102.859889,1230.500408,1824.424167,2011.181383,622.396982,...,1189.462895,1865.434031,1214.150237,943.167196,6,1,1,99,2,2019
3,14,294,3060.670617,2427.141134,2519.407191,2478.295578,1796.355480,2572.237530,2477.738650,2147.053226,...,2487.448270,1828.423286,1954.004295,3713.693577,6,1,1,99,2,2019
4,15,294,10908.362129,8043.579111,9314.455576,12369.290613,8117.648314,8700.781006,9736.936031,10220.615273,...,8171.207931,9054.016227,8956.806859,8997.675913,6,1,1,99,2,2019


In [3]:
index_cols = ['location_id','measure_id','metric_id','year_id','rei_id','sex_id','cause_id']

In [4]:
# sum over age groups
smok_sum = smok_attrib.groupby(index_cols).sum()
smok_sum = smok_sum.drop(columns=[c for c in smok_sum.columns if 'draw' not in c])
smok_sum

draw_0  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       1.981327e+06   
                                                       426       4.262681e+05   
                                                       493       3.407809e+05   
                                                       509       3.992591e+05   
                                                2      294       3.568805e+05   
                                                       426       6.738227e+04   
                                                       493       7.402235e+04   
                                                       509       1.001954e+05   

                                                                       draw_1  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       2.079242e+06   
                                                       426       4.457476e+05   
                                                       493       3.548579e+05   
                                                       509       4.380348e+05   
                                                2      294       3.107612e+05   
                                                       426       5.534052e+04   
                                                       493       6.676455e+04   
                                                       509       9.178476e+04   

                                                                      draw_10  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       1.906669e+06   
                                                       426       4.033320e+05   
                                                       493       3.200892e+05   
                                                       509       4.118919e+05   
                                                2      294       3.200611e+05   
                                                       426       6.071631e+04   
                                                       493       6.895170e+04   
                                                       509       8.700721e+04   

                                                                     draw_100  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       2.146907e+06   
                                                       426       4.597265e+05   
                                                       493       3.540967e+05   
                                                       509       4.725544e+05   
                                                2      294       3.434126e+05   
                                                       426       6.831723e+04   
                                                       493       7.423998e+04   
                                                       509       9.183636e+04   

                                                                     draw_101  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       1.952828e+06   
                                                       426       4.036931e+05   
                                                       493       3.228831e+05   
                                                       509       4.410513e+05   
                                                2      294       2.776932e+05   
                                                       426       5.928885e+04   
                                                       493       5.424660e+04   
                                                       509       7.507652e+04   

                

In [5]:
smok_modeled = smok_sum.reset_index().loc[smok_sum.reset_index().cause_id != 294]
smok_modeled['cause_id'] = 'modeled'
smok_modeled = smok_modeled.groupby(index_cols).sum()
smok_modeled

draw_0  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.166308e+06   
                                                2      modeled   2.416000e+05   

                                                                       draw_1  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.238640e+06   
                                                2      modeled   2.138898e+05   

                                                                      draw_10  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.135313e+06   
                                                2      modeled   2.166752e+05   

                                                                     draw_100  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.286378e+06   
                                                2      modeled   2.343936e+05   

                                                                     draw_101  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.167628e+06   
                                                2      modeled   1.886120e+05   

                                                                     draw_102  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.177982e+06   
                                                2      modeled   2.486018e+05   

                                                                     draw_103  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.219523e+06   
                                                2      modeled   2.241247e+05   

                                                                     draw_104  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.335566e+06   
                                                2      modeled   2.173065e+05   

                                                                     draw_105  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.137213e+06   
                                                2      modeled   1.788683e+05   

                                                                      draw_106  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                  
6           1          1         2019    99     1      modeled   971838.404964   
                                                2      modeled   183627.467344   

                                                                 ...  \
location_id measure_id metric_id year_id rei_id sex_id cause_id  ...   
6           1          1         2019    99     1      modeled   ...   
                                                2      modeled   ...   

                                                                     draw_990  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      modeled   1.143109e+06   
                                                2      modeled   2.692751e+05   

                                                                     draw_991  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    

In [6]:
smok_fin = smok_sum.append(smok_modeled)
smok_fin

draw_0  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       1.981327e+06   
                                                       426       4.262681e+05   
                                                       493       3.407809e+05   
                                                       509       3.992591e+05   
                                                2      294       3.568805e+05   
                                                       426       6.738227e+04   
                                                       493       7.402235e+04   
                                                       509       1.001954e+05   
                                                1      modeled   1.166308e+06   
                                                2      modeled   2.416000e+05   

                                                                       draw_1  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       2.079242e+06   
                                                       426       4.457476e+05   
                                                       493       3.548579e+05   
                                                       509       4.380348e+05   
                                                2      294       3.107612e+05   
                                                       426       5.534052e+04   
                                                       493       6.676455e+04   
                                                       509       9.178476e+04   
                                                1      modeled   1.238640e+06   
                                                2      modeled   2.138898e+05   

                                                                      draw_10  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       1.906669e+06   
                                                       426       4.033320e+05   
                                                       493       3.200892e+05   
                                                       509       4.118919e+05   
                                                2      294       3.200611e+05   
                                                       426       6.071631e+04   
                                                       493       6.895170e+04   
                                                       509       8.700721e+04   
                                                1      modeled   1.135313e+06   
                                                2      modeled   2.166752e+05   

                                                                     draw_100  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1          1         2019    99     1      294       2.146907e+06   
                                                       426       4.597265e+05   
                                                       493       3.540967e+05   
                                                       509       4.725544e+05   
                                                2      294       3.434126e+05   
                                                       426       6.831723e+04   
                                                       493       7.423998e+04   
                                                       509       9.183636e+04   
                                                1      modeled   1.286378e+06   
                                                2      modeled   2.343936e+05   

                                                                     draw_101  \
location_id measure_id metric_id year_id rei_id sex_id cause_id                 
6           1    

In [7]:
smok_all = smok_fin.reset_index()
smok_all = smok_all.loc[smok_all.cause_id==294].drop(columns='cause_id').set_index(index_cols[0:(len(index_cols) - 1)])
smok_all

draw_0  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.981327e+06   
                                                2       3.568805e+05   

                                                              draw_1  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       2.079242e+06   
                                                2       3.107612e+05   

                                                             draw_10  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.906669e+06   
                                                2       3.200611e+05   

                                                            draw_100  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       2.146907e+06   
                                                2       3.434126e+05   

                                                            draw_101  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.952828e+06   
                                                2       2.776932e+05   

                                                            draw_102  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.928905e+06   
                                                2       3.607163e+05   

                                                            draw_103  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       2.067454e+06   
                                                2       3.507068e+05   

                                                            draw_104  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       2.270499e+06   
                                                2       3.224745e+05   

                                                            draw_105  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.921219e+06   
                                                2       2.731816e+05   

                                                            draw_106  ...  \
location_id measure_id metric_id year_id rei_id sex_id                ...   
6           1          1         2019    99     1       1.665847e+06  ...   
                                                2       2.780722e+05  ...   

                                                            draw_990  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.909636e+06   
                                                2       3.628891e+05   

                                                            draw_991  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.955507e+06   
                                                2       4.025820e+05   

                                                            draw_992  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       1.740128e+06   
                                                2       3.384984e+05   

                                                            draw_993  \
location_id measure_id metric_id year_id rei_id sex_id                 
6           1          1         2019    99     1       2.039695e+06   
                                                2       3.509402e+05 

In [8]:
smok_frac = smok_fin / smok_all
smok_frac

draw_0  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.215143   
                                                       493       0.171996   
                                                       509       0.201511   
                                                       modeled   0.588650   
                                                2      294       1.000000   
                                                       426       0.188809   
                                                       493       0.207415   
                                                       509       0.280753   
                                                       modeled   0.676977   

                                                                   draw_1  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.214380   
                                                       493       0.170667   
                                                       509       0.210670   
                                                       modeled   0.595717   
                                                2      294       1.000000   
                                                       426       0.178081   
                                                       493       0.214842   
                                                       509       0.295355   
                                                       modeled   0.688277   

                                                                  draw_10  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.211538   
                                                       493       0.167879   
                                                       509       0.216027   
                                                       modeled   0.595443   
                                                2      294       1.000000   
                                                       426       0.189702   
                                                       493       0.215433   
                                                       509       0.271846   
                                                       modeled   0.676981   

                                                                 draw_100  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.214134   
                                                       493       0.164933   
                                                       509       0.220109   
                                                       modeled   0.599177   
                                                2      294       1.000000   
                                                       426       0.198936   
                                                       493       0.216183   
                                                       509       0.267423   
                                                       modeled   0.682542   

                                                                 draw_101  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.206722   
                                                       493 

In [9]:
smok_frac_fin = smok_frac.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)
smok_frac_fin

count  \
location_id measure_id metric_id year_id rei_id sex_id cause_id           
6           1          1         2019    99     1      294       1000.0   
                                                       426       1000.0   
                                                       493       1000.0   
                                                       509       1000.0   
                                                       modeled   1000.0   
                                                2      294       1000.0   
                                                       426       1000.0   
                                                       493       1000.0   
                                                       509       1000.0   
                                                       modeled   1000.0   

                                                                     mean  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.205529   
                                                       493       0.166102   
                                                       509       0.215190   
                                                       modeled   0.586821   
                                                2      294       1.000000   
                                                       426       0.188361   
                                                       493       0.197681   
                                                       509       0.302690   
                                                       modeled   0.688732   

                                                                      std  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       0.000000   
                                                       426       0.007814   
                                                       493       0.005301   
                                                       509       0.010656   
                                                       modeled   0.012247   
                                                2      294       0.000000   
                                                       426       0.011651   
                                                       493       0.012536   
                                                       509       0.032623   
                                                       modeled   0.028651   

                                                                      min  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.172881   
                                                       493       0.128422   
                                                       509       0.174004   
                                                       modeled   0.506328   
                                                2      294       1.000000   
                                                       426       0.137033   
                                                       493       0.142683   
                                                       509       0.204089   
                                                       modeled   0.528904   

                                                                     2.5%  \
location_id measure_id metric_id year_id rei_id sex_id cause_id             
6           1          1         2019    99     1      294       1.000000   
                                                       426       0.190225   
                                                       493       0.155548   
     

## Conclusion:
Modeling lung cancer, COPD, and IHD mortality will account for 
58.6% (UI: 56.0 - 60.8) of all smoking attributable mortality in China in 2019 among makes of all ages and 68.9% (UI: 62.6 - 74.0) for females.

# Repeat but combine sexes

In [11]:
index_cols = ['location_id','measure_id','metric_id','year_id','rei_id','cause_id']

In [13]:
# sum over age groups and sexes
smok_sum = smok_attrib.groupby(index_cols).sum()
smok_sum = smok_sum.drop(columns=[c for c in smok_sum.columns if 'draw' not in c])

smok_modeled = smok_sum.reset_index().loc[smok_sum.reset_index().cause_id != 294]
smok_modeled['cause_id'] = 'modeled'
smok_modeled = smok_modeled.groupby(index_cols).sum()

smok_fin = smok_sum.append(smok_modeled)

smok_all = smok_fin.reset_index()
smok_all = smok_all.loc[smok_all.cause_id==294].drop(columns='cause_id').set_index(index_cols[0:(len(index_cols) - 1)])

smok_frac = smok_fin / smok_all
smok_frac_fin = smok_frac.apply(pd.DataFrame.describe, percentiles=[0.025,0.975], axis=1)
smok_frac_fin

count      mean  \
location_id measure_id metric_id year_id rei_id cause_id                     
6           1          1         2019    99     294       1000.0  1.000000   
                                                426       1000.0  0.203203   
                                                493       1000.0  0.170379   
                                                509       1000.0  0.227023   
                                                modeled   1000.0  0.600605   

                                                               std       min  \
location_id measure_id metric_id year_id rei_id cause_id                       
6           1          1         2019    99     294       0.000000  1.000000   
                                                426       0.007370  0.169461   
                                                493       0.005388  0.134863   
                                                509       0.012047  0.185423   
                                                modeled   0.013904  0.509302   

                                                              2.5%       50%  \
location_id measure_id metric_id year_id rei_id cause_id                       
6           1          1         2019    99     294       1.000000  1.000000   
                                                426       0.188204  0.203951   
                                                493       0.158991  0.170761   
                                                509       0.206169  0.226060   
                                                modeled   0.569341  0.602029   

                                                             97.5%       max  
location_id measure_id metric_id year_id rei_id cause_id                      
6           1          1         2019    99     294       1.000000  1.000000  
                                                426       0.215776  0.228360  
                                                493       0.179466  0.196029  
                                                509       0.254060  0.296207  
                                                modeled   0.625095  0.650621

## Conclusion:
Modeling lung cancer, COPD, and IHD mortality will account for 
60% (UI: 56.9 - 62.5) of all smoking attributable mortality in China in 2019
across all sex and ages